In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.messages import HumanMessage
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams
import os
from datasets import load_from_disk
import pandas as pd

In [ ]:
ds = load_from_disk(r"C:\Users\Lenovo\Desktop\cnn_dailymail")
train_ds = ds["train"]

#sample and shuffle
sample_ds = train_ds.shuffle(seed=42).select(range(1000))

df = pd.DataFrame(sample_ds)
print(df.head(50))

df.to_csv(r"C:\Users\Lenovo\Desktop\cnn_train_sample.csv", index=False, encoding="utf-8")

In [ ]:
loader = CSVLoader(file_path=r"C:\Users\Lenovo\Desktop\cnn_train_sample.csv", source_column="article", encoding = "utf-8")


In [ ]:
data = loader.load()

print(len(data))

In [1]:
docs = []

In [ ]:
QDRANT_URL=""
QDRANT_API_KEY=""
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")
qdrant = QdrantVectorStore.from_documents(
    data,
    dense_embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    url=QDRANT_URL,
    prefer_grpc=True,
    api_key=QDRANT_API_KEY,
    collection_name="dailymail",
)